<a href="https://colab.research.google.com/github/MnCSSJ4x/VR-MiniProject/blob/main/VR3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
import torch 
from torch import optim, cuda
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
# from torchsummary import summary
import matplotlib.pyplot as plt
%matplotlib inline

import math 
from torch.utils.data import Dataset, TensorDataset, random_split
from torchvision import transforms
from tqdm import tqdm 
import tarfile
!pip install imutils

import matplotlib.pyplot as plt
import matplotlib
import argparse 
import joblib
import cv2
import os
import time 
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [89]:
cuda = cuda.is_available()
print(f'Train on gpu: {cuda}')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Train on gpu: False


### Dataset Used 
The dataset we are planning to use is the caltech 101 dataset.  It consist of about 40 to 800 images per category. Most categories have about 50 images. Collected in September 2003 by Fei-Fei Li, Marco Andreetto, and Marc'Aurelio Ranzato. The size of each image is roughly 300 x 200 pixels. 

In [117]:
# image_paths = list(paths.list_images('/kaggle/input/multi-class-weather-dataset/Multi-class Weather Dataset'))
# dataset = ImageFolder(root='/kaggle/input/multiclass-weather-dataset/Multi-class Weather Dataset', transform=transform)
image_paths = list(paths.list_images('/kaggle/input/multi-class-weather-dataset'))

In [123]:
data = []
labels = []
label_names = []
for image_path in image_paths:
    label = image_path.split(os.path.sep)[-2]
    if label == 'BACKGROUND_Google':
        continue

    image = cv2.imread(image_path)
     # Check if the image was loaded successfully
    if image is None:
        print(f"Error loading image: {image_path}")
        continue
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image = cv2.resize(image, (224, 224))

    data.append(image)
    label_names.append(label)
    labels.append(label)


Error loading image: /kaggle/input/multi-class-weather-dataset/Multi-class Weather Dataset/Rain/rain141.jpg
Error loading image: /kaggle/input/multi-class-weather-dataset/Multi-class Weather Dataset/Shine/shine131.jpg


In [124]:
import warnings
warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)            

In [125]:
data = np.array(data)
labels = np.array(labels)

Labels are in string format and we need to get it to number format 

In [126]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [127]:
(X, X_Validation , Y, Y_Validation) = train_test_split(data, labels, 
                                                    test_size=0.2,  
                                                    stratify=labels,
                                                    random_state=42)

(x_train, x_test, y_train, y_test) = train_test_split(X, Y, 
                                                    test_size=0.25, 
                                                    random_state=42)

print(f"x_train examples: {x_train.shape}\nx_test examples: {x_test.shape}\nX_Validation examples: {X_Validation.shape}")

x_train examples: (673, 224, 224, 3)
x_test examples: (225, 224, 224, 3)
X_Validation examples: (225, 224, 224, 3)


We need to do transformations on the images from this dataset as the model we are intending to use is the AlexNet model trained on ImageNet where the image is of size 224x224. We rescale the image as well as noramise it based on imagenet standards. Other than that since images per class is too low we also try some data augmentation on the train set explictly 

In [128]:
train_transform = transforms.Compose(
    [transforms.ToPILImage(),
	 transforms.Resize((224, 224)),
     transforms.RandomRotation((-30, 30)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])])
val_transform = transforms.Compose(
    [transforms.ToPILImage(),
	 transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])])

Custom Dataset class which allows us to apply transformations and create a dataset object to pass into dataloader 

In [158]:
# from torchvision.datasets import VisionDataset
# from PIL import Image

# class ImageDataset(VisionDataset):
#     def __init__(self, images, labels=None, transforms=None, loader=None):
#         super().__init__(None, transforms, None, None)
#         self.images = images
#         self.labels = labels
#         self.transforms = transforms
#         self.loader = loader  # Optional: specify a custom image loading function
        
#     def __len__(self):
#         return len(self.images)
    
#     def __getitem__(self, index):
#         img = self.loader(self.images[index]) if self.loader else self.default_loader(self.images[index])
#         label = self.labels[index] if self.labels is not None else None

#         if self.transforms:
#             img = self.transforms(img)

#         if label is not None:
#             return img, label
#         else:
#             return img

#     def default_loader(self, path):
#         return Image.open(path).convert('RGB')



from torchvision.datasets import VisionDataset
from PIL import Image
import numpy as np
from torchvision.transforms import ToPILImage

class ImageDataset(VisionDataset):
    def __init__(self, images, labels=None, transforms=None, loader=None):
        super().__init__(None, transforms, None, None)
        self.images = images
        self.labels = labels
        self.transforms = transforms
        self.loader = loader  # Optional: specify a custom image loading function
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img = self.images[index]
        
        # Check if img is a file path or a NumPy array
        if isinstance(img, str):  # Check if img is a file path
            img = self.loader(img) if self.loader else self.default_loader(img)
        elif isinstance(img, np.ndarray):  # Check if img is a NumPy array
            img = Image.fromarray(img)
        else:
            raise TypeError("Unsupported image type")

        label = self.labels[index] if self.labels is not None else None

        # Skip ToPILImage transform if img is already a PIL Image
        if isinstance(img, Image.Image):
            if self.transforms:
                for transform in self.transforms.transforms:
                    if not isinstance(transform, ToPILImage):
                        img = transform(img)
        else:
            if self.transforms:
                img = self.transforms(img)

        if label is not None:
            return img, label
        else:
            return img

    def default_loader(self, path):
        return Image.open(path).convert('RGB')




In [159]:
train_data = ImageDataset(x_train, y_train, train_transform)
val_data = ImageDataset(X_Validation, Y_Validation, val_transform)
test_data = ImageDataset(x_test, y_test, val_transform)
 
# dataloaders
trainloader = DataLoader(train_data, batch_size=16, shuffle=True)
valloader = DataLoader(val_data, batch_size=16, shuffle=True)
testloader = DataLoader(test_data, batch_size=16, shuffle=False)

### Pre-trained Model Alexnet 

In [160]:
model = models.alexnet(pretrained=True)

In [161]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [162]:
for param in model.parameters():
    param.requires_grad = False

In [163]:
model.classifier[6] = nn.Linear(in_features=4096,out_features=101,bias=True)

In [164]:
for param in model.classifier[6].parameters():
  param.requires_grad = True

In [165]:
model 

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [166]:
model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [167]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
# Define your loss function
criterion = nn.CrossEntropyLoss()

In [168]:
epochs = 10 

#### Training and Validation 

In [169]:
def fit(model, dataloader):
    print('Training')
    model.train()
    Train_loss = 0.0
    train_running_correct = 0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, torch.max(target, 1)[1])
        Train_loss += loss.item()
        _, prediction = torch.max(outputs.data, 1)
        train_running_correct += (prediction == torch.max(target, 1)[1]).sum().item()
        loss.backward()
        optimizer.step()
        
    Train_loss = Train_loss/len(dataloader.dataset)
    Train_acc = 100. * train_running_correct/len(dataloader.dataset)
    
    print(f"Train Loss: {Train_loss:.4f}, Train Acc: {Train_acc:.2f}")
    
    return Train_loss, Train_acc

In [170]:
#validation function
def validate(model, dataloader):
    print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, target = data[0].to(device), data[1].to(device)
            outputs = model(data)
            loss = criterion(outputs, torch.max(target, 1)[1])
            
            val_running_loss += loss.item()
            _, prediction = torch.max(outputs.data, 1)
            val_running_correct += (prediction == torch.max(target, 1)[1]).sum().item()
        
        Validation_loss = val_running_loss/len(dataloader.dataset)
        Validation_acc = 100. * val_running_correct/len(dataloader.dataset)
        print(f'Val Loss: {Validation_loss:.4f}, Val Acc: {Validation_acc:.2f}')
        
        return Validation_loss, Validation_acc

In [171]:
print(f"Training on {len(train_data)} examples, validating on {len(val_data)} examples")
Train_loss , Train_acc = [], []
Validation_loss , Validation_acc = [], []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    Validation_epoch_loss, Validation_epoch_acc = validate(model, valloader)
    train_epoch_loss, train_epoch_accuracy = fit(model, trainloader)
    Validation_loss.append(Validation_epoch_loss)
    Validation_acc.append(Validation_epoch_acc)
    Train_acc.append(train_epoch_accuracy)
    Train_loss.append(train_epoch_loss)


Training on 673 examples, validating on 225 examples
Epoch 1 of 10
Validating


15it [00:04,  3.25it/s]                        


Val Loss: 0.3053, Val Acc: 4.00
Training


43it [00:14,  2.95it/s]                        


Train Loss: 0.0574, Train Acc: 85.74
Epoch 2 of 10
Validating


...

Train Loss: 0.0585, Train Acc: 93.61
Epoch 10 of 10
Validating


15it [00:04,  3.45it/s]                        


Val Loss: 0.0987, Val Acc: 87.11
Training


43it [00:14,  3.06it/s]                        

Train Loss: 0.0521, Train Acc: 92.27


### Test

In [172]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, target = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == torch.max(target, 1)[1]).sum().item()

print('Accuracy of the network on test images: %0.3f %%' % (
    100 * correct / total))

Accuracy of the network on test images: 90.667 %
